In [2]:
import tensorflow as tf

import tensorflow_hub as hub
import pandas as pd
import numpy as np
import sklearn

from matplotlib import pyplot
import imblearn
import gensim
import nltk
from pathlib import Path

from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
df_date = pd.read_csv('/storage/data/ad_matched_queries_20190831__by_date.csv')
df_date.head(5)

,campaign_label,date,source_medium,campaign,ad_group,ad_matched_query,ad_cost,impressions,clicks,users,new_users,sessions,bounces,created_accounts,transactions,transaction_revenue
0,brand,2017-03-07,google / cpc,Brand Terms,Paperspace For Teams,paper space,0.31,4,1,1,1,1,0,0,0,0.0
1,brand,2017-03-07,google / cpc,Brand Terms,Paperspace For Teams,paperspace,1.06,28,5,4,3,4,2,0,0,0.0
2,brand,2017-03-08,google / cpc,Brand Terms,Paperspace For Teams,paperspace,0.94,24,3,1,0,1,0,0,0,0.0
3,brand,2017-03-09,google / cpc,Brand Terms,Paperspace For Teams,paperspace,1.70,50,4,2,2,2,1,0,0,0.0
4,brand,2017-03-10,google / cpc,Brand Terms,Paperspace For Teams,paperspace,4.57,49,7,5,4,9,0,1,1,6.0


In [3]:
REPORT_SOURCE = 'google/ads'
REPORT_ID = 'InsightFall2019 Ad Group Keyword 20191001'
FILENAME = 'aggregate_20190831.csv.gz'

PATH_DATA = Path('/') / 'storage' / 'data' / REPORT_SOURCE / REPORT_ID / FILENAME
df_cat = pd.read_csv(PATH_DATA)
df_cat.head(5)


,Campaign ID,Ad group ID,Keyword ID,Campaign,Ad group,Search keyword,Keyword/Placement,Search keyword match type,Campaign type,Campaign bid strategy type,...,Currency,Cost,Impressions,Clicks,Conversions,Conv. value,Avg. position,Impr. (Abs. Top) %,Impr. (Top) %,Search Impr. share
0,765732120,39460828726,44160165835,General Product Terms,Cloud Virtual Desktop - Enterprise Market,+Cloud +Virtual +Desktop,+Cloud +Virtual +Desktop,Broad,Search Only,Target CPA,...,USD,0.0,4,0,0.0,0.0,1.5,0.0,0.0,0.3636
1,765732120,39616922879,341042909967,General Product Terms,VDI Cloud,+vdi in the +cloud,+vdi in the +cloud,Broad,Search Only,Target CPA,...,USD,0.0,1,0,0.0,0.0,7.0,0.0,0.0,1.0000
2,765732120,43344958467,27994524387,General Product Terms,Cloud Virtual Machine - Enterprise Market,+Cloud +Virtual +Machine,+Cloud +Virtual +Machine,Broad,Search Only,Target CPA,...,USD,0.0,2,0,0.0,0.0,2.0,0.0,0.0,1.0000
3,766802375,39100138983,28912106981,Competitor Terms,VMware Cloud Computing,+cloud +vmware,+cloud +vmware,Broad,Search Only,Enhanced CPC,...,USD,0.0,1,0,0.0,0.0,3.0,0.0,0.0,0.5000
4,765732120,39616922879,32957936454,General Product Terms,VDI Cloud,+vdi +cloud,+vdi +cloud,Broad,Search Only,Target CPA,...,USD,0.0,3,0,0.0,0.0,1.0,0.0,0.0,1.0000


In [5]:
import pickle
# bring in market research data
df_market = pd.read_csv('/storage/data/keyword_research/ps_ml_keyword_stats_20190423_United States_with_seed_links.csv')
market_keywords = df_market['Keyword']
pickle_out = open("market_keywords.pickle","wb")
pickle.dump(market_keywords, pickle_out)
pickle_out.close()
df_market.tail(5)

,_competitor,_seedLink,Keyword,Avg. monthly searches,Competition,Competition (indexed value),Top of page bid (low range),Top of page bid (high range),Ad impression share,Organic impression share,...,Jun 2018,Jul 2018,Aug 2018,Sep 2018,Oct 2018,Nov 2018,Dec 2018,Jan 2019,Feb 2019,Mar 2019
13944,weights & biases,https://www.wandb.com/,wandb,50.0,Low,0.0,NaN,NaN,NaN,NaN,...,90.0,90.0,140.0,110.0,110.0,140.0,110.0,110.0,140.0,170.0
13945,weights & biases,https://www.wandb.com/,weight and bias in machine learning,10.0,Low,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,10.0,10.0,10.0,10.0,10.0
13946,weights & biases,https://www.wandb.com/,weight bias,210.0,Low,0.0,NaN,NaN,NaN,NaN,...,260.0,210.0,210.0,320.0,480.0,390.0,170.0,390.0,320.0,480.0
13947,weights & biases,https://www.wandb.com/,weights and biases,50.0,Low,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,320.0,320.0,260.0,390.0,390.0,480.0
13948,weights & biases,https://www.wandb.com/,weights and biases machine learning,10.0,Low,24.0,28.9,41.49,NaN,NaN,...,20.0,20.0,30.0,10.0,30.0,10.0,10.0,10.0,10.0,10.0


In [5]:
#corpus = [df_date["ad_matched_query"].str,df_market['Keyword'].str, df_cat['Keyword/Placement']]
#corpus = ' '.join(map(str,[df_date["ad_matched_query"].str,df_market['Keyword'].str, df_cat['Keyword/Placement']] ))
#corpus
df_append = df_date['ad_matched_query'].append([df_cat['Keyword/Placement'], df_market['Keyword']])
type(df_append)

pandas.core.series.Series

In [6]:
#cleaning 
df_append.str.lower().str.replace('[!"#$%&()*+-/:;<=>?@[\\]^_`{|}~]',' ')
df_append.str.replace("[0-9]", ' ')
df_append.dropna()
df_append = df_append.apply(lambda x:' '.join(x.split()))
unique_words = df_append.unique()
print(len(unique_words))


39905


In [7]:
## stem and lowercase
from nltk.stem import PorterStemmer
ps = PorterStemmer()

words_stemmed = [ps.stem(unique_words) for unique_words in unique_words]
#print(words_stemmed)

In [8]:
words_array = np.asarray(words_stemmed)
words_array.shape

(39905,)

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(unique_words, test_size=.5)
print(train.size)
print(test.size)

19952
19953


In [10]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [11]:
# ELMO test
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

In [12]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [13]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [14]:
import tqdm

for ii in tqdm.trange(100):
    elmo_train = [elmo(train) for train in list_train]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
elmo_train = [elmo_vectors(train) for train in list_train]

In [ ]:
elmo_test = [elmo_vectors(test) for test in list_test]

In [15]:

input_text = layers.Input(shape=(1,), dtype="string")
embedding = ElmoEmbeddingLayer()(input_text)
dense = layers.Dense(64, activation='relu')(embedding)
#dropout = layers.Dropout(0.3)(dense)
pred = layers.Dense(50, activation='softmax')(dense)#(dropout)

model = Model(inputs=[input_text], outputs=pred)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

W1010 04:50:17.951891 140320610342720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1010 04:50:17.953347 140320610342720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1010 04:50:19.592317 140320610342720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1010 04:50:19.619011 140320610342720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1010 04:50:19.642912 140320610342720 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3250      
Total params: 68,854
Trainable params: 68,854
Non-trainable params: 0
_________________________________________________________________
